# Glacier Gallery

## Import packages

In [ ]:
import panel as pn
import os, numpy as np, pandas as pd, cartopy.crs as ccrs, bokeh
import holoviews as hv, geoviews as gv, datashader as ds
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, Plot, Range1d, Text, TapTool, CustomJS, Legend, LegendItem, RadioGroup
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_file, show, output_notebook, reset_output
from bokeh.models.markers import Circle
hv.extension('bokeh', width=100)
pn.extension()

## Choose language for html document!

In [ ]:
language = 'en'
file_name = 'gallery_en'
#language = 'de'
#file_name = 'gallery_de'

### Import text in different languages and selection of glacier with meta data

In [ ]:
# import all texts in the corresponding language from here:
from international import trads, supported_languages, links
from Glaciers import glaciers

### Function to prepare coordinates

In [ ]:
import math
from ast import literal_eval

def merc(Coords):
    lat = Coords[0]
    lon = Coords[1]

    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale    
    return (x, y)

## Load data

In [ ]:
df_all_all = pd.read_hdf('./data/rgi62_era5_itmix_df.h5', 'df')
df_all_all['rgi_id']= list(df_all_all.axes[0])
df_all = df_all_all[['rgi_id', 'CenLon', 'CenLat', 'Lmax', 'GlacierType','TerminusType', 'GlacierStatus', 'IsTidewater', 'IsNominal' ]].copy()

## Select glaciers, add photos and build lists with information about them

In [ ]:
Glacier_No = glaciers # imported glaciers and meta data from Glaciers.py

# Create lists from glacier list above
glacier_list = []
glacier_names = []
gltype = []
glacier_pics =[]
glpic_pretext_source = []
glpic_cite = []
glpic_lic = []
glpic_wi = []
glpic_he = []
glpic_src = []
for gl_no in Glacier_No:
    glacier_list.append(gl_no[0])
    glacier_names.append(gl_no[1])
    gltype.append(gl_no[2])
    glacier_pics.append(gl_no[3])
    glpic_pretext_source.append(gl_no[4])
    glpic_cite.append(gl_no[5])
    glpic_lic.append (gl_no[6])
    glpic_wi.append(gl_no[8])
    glpic_he.append(gl_no[9])
    glpic_src.append(gl_no[10])

# create a new dataframe with the selected glaciers from the loaded data about glaciers 
df = df_all.loc[df_all['rgi_id'].isin(glacier_list)].copy()
df['name'] = glacier_names
df['Coords'] = list(zip(round(df.CenLat,2), round(df.CenLon,2)))
df['type'] = gltype
df['img'] = glacier_pics
df['pic_pretext'] = glpic_pretext_source
df['glpic_lic'] = glpic_lic
df['pic_cite'] = glpic_cite
df['pic_width'] = glpic_wi
df['pic_height'] = glpic_he
df['pic_src'] = glpic_src

## Define texts in according language

In [ ]:
title                 = trads['Title_hp'][language]
instruction           = trads['instruction'][language]
link_to_license_intro = trads['link_to_license_intro'][language]
links_pic_src_intro   = trads['links_pic_src_intro'][language]

# tab titles
tab_title1 = trads['tab_title_1'][language]
tab_title2 = trads['tab_title_2'][language]

links_to_license = links['links_license'] 
links_pic_src    = links['links_pics']

df['Def'] = trads['Glacier_definition'][language]
df['Title'] = trads['Glacier_title'][language]
df['text_source'] = trads['Glacier_text_source'][language]

# translate "terminating" entry:
df.loc[df['TerminusType'] == 'Land-terminating', 'TerminusType']   = trads['TerminusType_land'][language]
df.loc[df['TerminusType'] == 'Marine-terminating', 'TerminusType'] = trads['TerminusType_marine'][language]

# define colors for dots of different glacier types
df.loc[df['type'] == 'Pure ice glacier', 'color']       = '#85a9c1'
df.loc[df['type'] == 'Calving glacier', 'color']        = '#FD90A7'
df.loc[df['type'] == 'Debris covered glacier', 'color'] = '#9FBC40'
df.loc[df['type'] == 'Ice cap', 'color']                = '#FFFED9'

### Add texts about glacier types to the dataframe

In [ ]:
df.loc[df['type'] == 'Pure ice glacier',       'glaciertype_text'] = trads['pure_ice_text'][language]
df.loc[df['type'] == 'Calving glacier',        'glaciertype_text'] = trads['calving_text'][language]
df.loc[df['type'] == 'Debris covered glacier', 'glaciertype_text'] = trads['debris_text'][language]
df.loc[df['type'] == 'Ice cap',                'glaciertype_text'] = trads['ice_cap_text'][language]
df.loc[df['type'] == 'Pure ice glacier',       'glaciertype_source'] = trads['pure_ice_source'][language]
df.loc[df['type'] == 'Calving glacier',        'glaciertype_source'] = trads['calving_glacier_source'][language]
df.loc[df['type'] == 'Debris covered glacier', 'glaciertype_source'] = trads['debris_glacier_source'][language]
df.loc[df['type'] == 'Ice cap',                'glaciertype_source'] = trads['ice_cap_source'][language]

# translate "type" entry:
df.loc[df['type'] == 'Pure ice glacier', 'type']       = trads['type_pure_ice'][language]
df.loc[df['type'] == 'Calving glacier', 'type']        = trads['type_calving'][language]
df.loc[df['type'] == 'Debris covered glacier', 'type'] = trads['type_debris_covered'][language]
df.loc[df['type'] == 'Ice cap', 'type']                = trads['type_ice_cap'][language]

# 3 empty columns for changing text box
df['empty1'] = ''
df['empty2'] = ''
df['empty3'] = ''

## Build special datastructure with all information

In [ ]:
# prepare coordinates  
df.loc[:, 'coords_x'] = df['Coords'].apply(lambda x: merc(x)[0])
df.loc[:, 'coords_y'] = df['Coords'].apply(lambda x: merc(x)[1])

# build source
source = ColumnDataSource(df)

## Design for map with hover and webpage

In [ ]:
# Hover
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@img" 
                height="@pic_height" 
                alt="image loading..." 
                width="@pic_width"
                style=" margin: 2px 2px 0px 2px;"
                border="2">
            </img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@name                </span> <br>
            <span style="font-size: 15px;                   ">@type                </span> <br>
            <span style="font-size: 15px;                   ">Length:              </span>
            <span style="font-size: 15px;                   ">@Lmax                </span>
            <span style="font-size: 15px;                   ">m                    </span> <br>
            <span style="font-size: 15px;                   ">@TerminusType        </span> <br>
            <span style="font-size: 15px;                   ">Location             </span>
            <span style="font-size: 13px; color: #696;      ">(@CenLat, @CenLon)   </span> <br>
            <span style="font-size: 9px;                    ">@pic_pretext @pic_cite @glpic_lic </span>
        </div>
    </div>
"""
hover = HoverTool()
hover.tooltips = TOOLTIPS

############################################################################
# Create figure for the map
p = figure(x_axis_type="mercator", 
           y_axis_type="mercator",
           plot_width = 1000,
           plot_height = 650,
           background='black',
           tools=['tap', 'wheel_zoom', 'reset', 'pan'])

############################################################################
# Design figure
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False
p.xgrid.grid_line_color = None

############################################################################
# add map
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)
# add glacier dots
p.circle(x='coords_x',
         y='coords_y', 
         source=source,
         size=10,
         color='color',
         line_color="black", 
         fill_alpha=0.9,
         legend='type'
        )
# add markers around the dots when mouse is directly over glacier dots (belongs to hover)
cr = p.circle(x='coords_x', y='coords_y', source=source, size=20,
                fill_color="white", hover_fill_color="blue",
                fill_alpha=0.0, hover_alpha=0.3,
                line_color=None, hover_line_color="white")
# add hover
hover = HoverTool(tooltips=TOOLTIPS, renderers=[cr])
p.add_tools(hover)

############################################################################
# build text boxes 

# glacier type text box
def construct_changing_text_box(source): 
    # Plot and axes                                                             
    xdr = Range1d(0, 1220)                                                       
    ydr = Range1d(0, 1120)                                                       

    plot = Plot(                                        
        x_range=xdr, y_range=ydr,                                   
        plot_height=650, min_border=0, 
        plot_width=380
        )                                                                           
    # Add the writing 
    title = Text(x=10, y=1055, text='empty1', text_color="#85a9c1", text_alpha=0.2, text_font='helvetica', text_font_size='14pt')
    text  = Text(x=0,  y=250,  text='empty2', text_color="#326a86", text_alpha=0.2, text_font='helvetica', text_font_size='10pt')
    sourc = Text(x=10, y=0,    text='empty3', text_color="#85a9c1", text_alpha=0.2, text_font='helvetica', text_font_size='7pt')
    
    plot.add_glyph(source, text)
    plot.add_glyph(source, title)    
    plot.add_glyph(source, sourc)      
    return plot

# Glacier definition text box
def construct_text_box(source): 
    # Plot and axes                                                             
    xdr = Range1d(0, 1220)                                                       
    ydr = Range1d(0, 1120)                                                       

    plot = Plot(                                        
        x_range=xdr, y_range=ydr,                                   
        plot_height=650, min_border=0, 
        plot_width=380
        )                                                                           
    # Add the writing 
    title1 = Text(x=10, y=1055, text='Title',       text_color="#85a9c1", text_alpha=0.2, text_font='helvetica', text_font_size='14pt')
    text1  = Text(x=0,  y=250,  text='Def',         text_color="#326a86", text_alpha=0.2, text_font='helvetica', text_font_size='10pt')
    cite1  = Text(x=10, y=0,    text='text_source', text_color="#85a9c1", text_alpha=0.2, text_font='helvetica', text_font_size='7pt')

    plot.add_glyph(source, title1)
    plot.add_glyph(source, text1)
    plot.add_glyph(source, cite1)
    return plot


JS_CODE ="""
    // Get index of current selected datapoint
    sel_point_index = cb_data.source.attributes.selected["1d"]["indices"][0];     

    /* replace all name_display values with the name_label
       value of currently selected point */
       
    for (i=0; i < cb_data.source.data.type.length; i++) {
        cb_data.source.data.empty1[i] = cb_data.source.data.type[sel_point_index];
        }       
    cb_data.source.change.emit();
    
    for (i=0; i < cb_data.source.data.type.length; i++) {
        cb_data.source.data.empty2[i] = cb_data.source.data.glaciertype_text[sel_point_index];
        }      
     cb_data.source.change.emit();

    for (i=0; i < cb_data.source.data.type.length; i++) {
        cb_data.source.data.empty3[i] = cb_data.source.data.glaciertype_source[sel_point_index];
        }      
     cb_data.source.change.emit();
    """

newtaptool = p.select(type=TapTool)
newtaptool.callback = CustomJS(code=JS_CODE)
changing_text_box = construct_changing_text_box(source)
text_box = construct_text_box(source)

# make 2 tabs (one for each box)
from bokeh.models.widgets import Panel, Tabs
tab1 = Panel(child=changing_text_box, title=tab_title1)
tab2 = Panel(child=text_box, title=tab_title2)
tabs = Tabs(tabs=[ tab1, tab2 ])

#### Logos

In [ ]:
oggm_logo   = '<a href="http://edu.oggm.org"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220></a>'
pn_logo     = '<a href="https://panel.pyviz.org"><img src="http://panel.pyviz.org/_static/logo_stacked.png" width=60></a>'
fk_logo     = '<a href="https://www.uibk.ac.at/foerderkreis1669/"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/uibk_logo_narrow.png" width=140></a>'

## Design application

In [ ]:
# Panel
header = pn.Row(pn.Pane(oggm_logo),  pn.layout.Spacer(width=30), 
                pn.Column(pn.Row(pn.Spacer(width=130), pn.Pane(title, width=400)), pn.Row(pn.Spacer(width=80), pn.Pane(instruction, width=1200))),
                pn.layout.HSpacer()
                )
logos = pn.Column(pn.Spacer(height=30), pn.Pane(pn_logo, width=100), pn.Spacer(height=40), pn.Pane(fk_logo))
links = pn.Column(link_to_license_intro, links_to_license, links_pic_src_intro, links_pic_src)

# Put plot and text together and show it
app = pn.Column(header, pn.Row(logos, pn.Column(pn.Spacer(height=22), p), pn.Spacer(width=2), tabs), pn.Spacer(height=60), links, width_policy='max', height_policy='max')
#app.show()

In [ ]:
# put transform app to one model:
app_to_save = app.get_root()

from bokeh.resources import CDN
from bokeh.embed import file_html
# save model in html
bokeh.io.save(app_to_save, file_name, CDN)